# Trimestral



In [17]:
pip install fredapi

In [18]:
from fredapi import Fred
from datetime import datetime
from scipy import stats
from statsmodels.sandbox.regression import gmm
from google.colab import files

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [19]:
#######################################
#### Importando os dados do Github ####
#######################################


# Importando os dados do Github
df_q  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/BEA_Tab115_Quarter.csv',
                             delimiter = ";")

p_q  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/BEA_Tab114_Quarter.csv',
                             delimiter = ";")

# Deletando a primeira coluna
del df_q['Line']
del p_q['Line']

# Transpondo a matriz
df_q  = df_q.T
p_q   = p_q.T

# Excluindo primeira linha a add ela como var name
df_q.columns = df_q.iloc[0, :]
p_q.columns  = p_q.iloc[0, :]

df_q = df_q.iloc[1:, :]
p_q  = p_q.iloc[1:, :]



# Removendo o underline
df_q.index = df_q.index.str.replace("_", "")
p_q.index = p_q.index.str.replace("_", "")


# Transformando em data
df_q.index = pd.to_datetime( df_q.index , errors='coerce')
p_q.index = pd.to_datetime( p_q.index , errors='coerce')

# Renomeando
df_q = df_q.rename(columns={df_q.columns[0]: 'pib', 
                            df_q.columns[2]: 'bens', 
                            df_q.columns[3]: 'duraveis', 
                            df_q.columns[4]: 'n_duraveis', 
                            df_q.columns[5]: 'servicos'})

p_q  = p_q.rename(columns={p_q.columns[0]: 'pib',
                           p_q.columns[2]: 'bens', 
                           p_q.columns[3]: 'duraveis', 
                           p_q.columns[4]: 'n_duraveis', 
                           p_q.columns[5]: 'servicos'})

# Transformando em .float
df_q = df_q[['pib', 'bens', 'duraveis', 'n_duraveis', 'servicos']].apply(pd.to_numeric, errors='coerce')
p_q = p_q[['pib', 'bens', 'duraveis', 'n_duraveis', 'servicos']].apply(pd.to_numeric, errors='coerce')

# Calculando consumo real, separando duráveis e não duraveis NAO TEM SERVICO
pib_real = df_q['pib']/p_q['pib']
duraveis_real = df_q['duraveis']/p_q['duraveis']
consumo_real = df_q['n_duraveis']/p_q['n_duraveis']
servicos_real = df_q['servicos']/p_q['servicos']

# Crescimento real do consumo
cresc_duravel_real  = duraveis_real/duraveis_real.shift(1) - 1
cresc_consumo_real  = consumo_real/consumo_real.shift(1) - 1
cresc_servicos_real = servicos_real/servicos_real.shift(1) - 1

# Calculando a tx de inflação (1+...%) = taxa de inflação de todos os bens na economia
tx_inf1 = p_q['bens']/p_q['bens'].shift(1)


dta = pd.DataFrame([pib_real,
                    consumo_real, cresc_consumo_real,  p_q['n_duraveis'], 
                    duraveis_real, cresc_duravel_real, p_q['duraveis'], 
                    servicos_real, cresc_servicos_real, p_q['servicos'],
                    tx_inf1]).T

dta.columns = ['pib',
               'c', 'c_growth', 'pc', 
               'k', 'k_growth', 'pk', 
               's', 's_growth', 'ps', 
               'inflacao']

In [20]:
dta

,pib,c,c_growth,pc,k,k_growth,pk,s,s_growth,ps,inflacao
1947-01-01,20316.150054,5157.184185,NaN,14.518,445.634409,NaN,46.500,7700.826446,NaN,7.865,NaN
1947-04-01,20274.315859,5245.719353,0.017167,14.659,454.973577,0.020957,46.928,7812.721295,0.014530,7.908,1.009585
1947-07-01,20251.947420,5269.967837,0.004623,14.924,461.326552,0.013963,47.190,7795.973817,-0.002144,8.097,1.015302
1947-10-01,20652.381331,5200.975610,-0.013092,15.375,491.329359,0.065036,47.805,7754.981282,-0.005258,8.281,1.026329
1948-01-01,20888.382330,5218.276061,0.003326,15.627,488.831223,-0.005084,48.170,7837.555529,0.010648,8.329,1.014375
...,...,...,...,...,...,...,...,...,...,...,...
2019-10-01,192473.550523,30181.122270,-0.001879,99.673,18118.905496,0.007707,85.774,85845.455081,0.005082,118.781,1.000538
2020-01-01,190167.040042,30704.753677,0.017350,99.544,17521.532445,-0.032970,85.406,83650.390102,-0.025570,119.456,0.997701
2020-04-01,172958.656743,29478.355554,-0.039942,97.808,17447.791141,-0.004209,84.727,73066.493811,-0.126525,119.560,0.985719
2020-07-01,185968.235563,31543.796766,0.070066,98.763,20284.656330,0.162592,86.420,79192.365842,0.083840,120.380,1.013309


In [21]:
#######################################
###### Importando dados do SP500 ######
#######################################

sp500  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/sp500.csv',
                             delimiter = ",")

# Selecionando variaveis
sp500 = sp500[['ref.date', 'price.adjusted']]
sp500.columns = ['date', 'price']

# Adicionando o indice de datas
sp500.index = pd.to_datetime( sp500['date'] , errors='coerce')


# Criando variavel indicadora para mudança de quarter
sp500['quarter'] = sp500.index.quarter
sp500['year'] = sp500.index.year
sp500['group_indicator'] = sp500['quarter']  != sp500['quarter'] .shift(-1)

# Filtrando apenas os ultimos meses do quarter e selecionando ano que quero
sp500 = sp500[sp500['group_indicator'] == True]
sp500 = sp500[sp500['year'] >= 1947][:-2]

# Selecionando variaveis
sp500 = sp500[['price']]


# Copiando o indice
sp500.index = p_q.index

# Calculando o retonro e o ret real
sp500['sp500_ret_real'] = ((sp500['price']/sp500['price'].shift(1)) / tx_inf1) - 1
sp500['sp500_ret'] = sp500['price']/sp500['price'].shift(1) - 1

# Fazendo o merge dos dois datasets
dta = dta.join(sp500)

In [22]:
fred = Fred(api_key='548624ed839059b759900a51992bc8c8')

# Obtendo dados trimestrais do FRED usando a média do período
tb3m = fred.get_series('DTB3', frequency = 'q', aggregation_method = 'avg')

# Transformando em dataframe
tb3m = pd.DataFrame(tb3m, columns = ['T-Bill'])

# Transformando o retorno em trimestral
tb3m['TBill_ret'] = np.power(1+tb3m['T-Bill']/100, 0.25) - 1

In [23]:
# Fazendo o merge dos dois datasets
dta2 = dta.join(tb3m['TBill_ret'])

dta2['TBill_ret_real'] = ( 1 + dta2['TBill_ret'] ) / dta2['inflacao'] - 1

dta2 = dta2[['pib',
             'c', 'c_growth', 'pc', 
             'k', 'k_growth', 'pk', 
             's', 's_growth', 'ps', 
             'sp500_ret_real', 'TBill_ret_real']].dropna()

In [24]:
dta2.head()

,pib,c,c_growth,pc,k,k_growth,pk,s,s_growth,ps,sp500_ret_real,TBill_ret_real
1954-01-01,26456.917267,6032.108179,0.009646,16.787,612.575385,-0.030679,53.724,9916.723952,0.009418,10.183,0.054588,-0.001841
1954-04-01,26461.143092,6016.150101,-0.002646,16.842,642.405728,0.048697,52.375,10086.739194,0.017144,10.203,0.116224,0.005776
1954-07-01,26819.123397,6092.160954,0.012634,16.775,645.100551,0.004195,51.516,10260.639859,0.017241,10.221,0.036534,0.009413
1954-10-01,27382.792163,6194.585854,0.016813,16.697,684.643171,0.061297,51.215,10384.607891,0.012082,10.265,0.137278,0.007692
1955-01-01,28190.336450,6259.103977,0.010415,16.696,745.842631,0.089389,51.535,10506.538797,0.011742,10.323,0.082005,0.001627


In [25]:
# Exportanto dados
dta2.to_csv('dataset_trim.csv')
files.download('dataset_trim.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Anual

In [26]:
#######################################
#### Importando os dados do Github ####
#######################################


# Importando os dados do Github
df_q  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/BEA_Tab115_Annual.csv',
                             delimiter = ";")

p_q  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/BEA_Tab114_Annual.csv',
                             delimiter = ";")

# Deletando a primeira coluna
del df_q['Line']
del p_q['Line']

# Transpondo a matriz
df_q  = df_q.T
p_q   = p_q.T

# Excluindo primeira linha a add ela como var name
df_q.columns = df_q.iloc[0, :]
p_q.columns  = p_q.iloc[0, :]

df_q = df_q.iloc[1:, :]
p_q  = p_q.iloc[1:, :]



# Removendo o underline
df_q.index = df_q.index.str.replace("_", "")
p_q.index = p_q.index.str.replace("_", "")


# Transformando em data
df_q.index = pd.to_datetime( df_q.index , errors='coerce')
p_q.index = pd.to_datetime( p_q.index , errors='coerce')


# Renomeando
df_q = df_q.rename(columns={df_q.columns[0]: 'pib', 
                            df_q.columns[2]: 'bens', 
                            df_q.columns[3]: 'duraveis', 
                            df_q.columns[4]: 'n_duraveis', 
                            df_q.columns[5]: 'servicos'})

p_q  = p_q.rename(columns={p_q.columns[0]: 'pib', 
                           p_q.columns[2]: 'bens', 
                           p_q.columns[3]: 'duraveis', 
                           p_q.columns[4]: 'n_duraveis', 
                           p_q.columns[5]: 'servicos'})


# Transformando em .float
df_q = df_q[['pib', 'bens', 'duraveis', 'n_duraveis', 'servicos']].apply(pd.to_numeric, errors='coerce')
p_q = p_q[['pib', 'bens', 'duraveis', 'n_duraveis', 'servicos']].apply(pd.to_numeric, errors='coerce')


# Calculando consumo real, separando duráveis e não duraveis NAO TEM SERVICO
pib_real       = df_q['pib']/p_q['pib']
duraveis_real  = df_q['duraveis']/p_q['duraveis']
consumo_real   = df_q['n_duraveis']/p_q['n_duraveis']
servicos_real  = df_q['servicos']/p_q['servicos']


# Crescimento real do consumo
cresc_duravel_real  = duraveis_real/duraveis_real.shift(1) - 1
cresc_consumo_real  = consumo_real/consumo_real.shift(1) - 1
cresc_servicos_real = servicos_real/servicos_real.shift(1) - 1


# Calculando a tx de inflação (1+...%) = taxa de inflação de todos os bens na economia
tx_inf1 = p_q['bens']/p_q['bens'].shift(1)


dta = pd.DataFrame([pib_real,
                    consumo_real, cresc_consumo_real,  p_q['n_duraveis'], 
                    duraveis_real, cresc_duravel_real, p_q['duraveis'], 
                    servicos_real, cresc_servicos_real, p_q['servicos'],
                    tx_inf1]).T

dta.columns = ['pib',
               'c', 'c_growth', 'pc', 
               'k', 'k_growth', 'pk', 
               's', 's_growth', 'ps', 
               'inflacao']

In [27]:
dta.head(n=10)

,pib,c,c_growth,pc,k,k_growth,pk,s,s_growth,ps,inflacao
1929-01-01,11126.529744,3374.863279,NaN,10.057,306.514485,NaN,32.067,4951.097363,NaN,6.789,NaN
1930-01-01,10201.461147,3199.056109,-0.052093,9.535,253.759523,-0.172112,30.190,4848.650288,-0.020692,6.594,0.946788
1931-01-01,9506.326004,3164.622179,-0.010764,8.152,219.283276,-0.135862,26.956,4724.796085,-0.025544,6.130,0.862120
1932-01-01,8250.901026,2885.820575,-0.088099,6.989,166.687718,-0.239852,23.751,4439.689419,-0.060343,5.538,0.861552
1933-01-01,8142.755378,2875.593269,-0.003544,6.953,162.337102,-0.026100,23.174,4275.337187,-0.037019,5.190,0.991688
1934-01-01,9076.086957,3109.998700,0.081516,7.691,185.785232,0.144441,24.566,4484.387198,0.048897,5.124,1.098651
1935-01-01,9888.252531,3288.248616,0.057315,7.948,224.874115,0.210398,24.427,4641.902215,0.035125,5.236,1.026833
1936-01-01,11158.905551,3663.615848,0.114154,7.976,274.718118,0.221653,24.567,4927.269310,0.061476,5.321,1.003928
1937-01-01,11803.909126,3753.786502,0.024612,8.253,289.026154,0.052083,25.579,5157.818445,0.046790,5.519,1.035982
1938-01-01,11294.543574,3808.206958,0.014497,7.847,239.553991,-0.171168,25.560,5116.944243,-0.007925,5.524,0.959438


In [28]:
#######################################
###### Importando dados do SP500 ######
#######################################

sp500  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/sp500.csv',
                             delimiter = ",")

# Selecionando variaveis
sp500 = sp500[['ref.date', 'price.adjusted']]
sp500.columns = ['date', 'price']

# Adicionando o indice de datas
sp500.index = pd.to_datetime( sp500['date'] , errors='coerce')


# Criando variavel indicadora para mudança de ano
sp500['quarter'] = sp500.index.quarter
sp500['month'] = sp500.index.month
sp500['year'] = sp500.index.year
sp500['group_indicator'] = sp500['month']  == 1

# Filtrando apenas os ultimos meses do quarter e selecionando ano que quero
sp500 = sp500[sp500['group_indicator'] == True]
sp500 = sp500[sp500['year'] >= 1929][:-1]

# Selecionando variaveis
sp500 = sp500[['price']]


# Copiando o indice
sp500.index = p_q.index

# Calculando o retonro e o ret real
sp500['sp500_ret_real'] = ((sp500['price']/sp500['price'].shift(1)) / tx_inf1) - 1
sp500['sp500_ret'] = sp500['price']/sp500['price'].shift(1) - 1

# Fazendo o merge dos dois datasets
dta = dta.join(sp500)

In [29]:
fred = Fred(api_key='548624ed839059b759900a51992bc8c8')

# Obtendo dados trimestrais do FRED usando a média do período
tb3m = fred.get_series('DTB3', frequency = 'a', aggregation_method = 'avg')

# Transformando em dataframe
tb3m = pd.DataFrame(tb3m, columns = ['T-Bill'])

# Transformando o retorno em trimestral
tb3m['TBill_ret'] = tb3m['T-Bill']/100

In [30]:
# Fazendo o merge dos dois datasets
dta2 = dta.join(tb3m['TBill_ret'])

dta2['TBill_ret_real'] = ( 1 + dta2['TBill_ret'] ) / dta2['inflacao'] - 1

dta2 = dta2[['pib',
             'c', 'c_growth', 'pc', 
             'k', 'k_growth', 'pk', 
             's', 's_growth', 'ps', 
             'sp500_ret_real', 'TBill_ret_real']].dropna()

In [31]:
dta2.head(n=10)

,pib,c,c_growth,pc,k,k_growth,pk,s,s_growth,ps,sp500_ret_real,TBill_ret_real
1954-01-01,26781.114997,6083.636364,0.015872,16.775,645.756316,-0.004260,52.207,10162.654140,0.034690,10.218,-0.056324,0.013250
1955-01-01,28767.951318,6399.435977,0.051910,16.666,783.482357,0.213279,52.005,10671.735155,0.050093,10.391,0.481662,0.023219
1956-01-01,29377.157427,6624.371488,0.035149,16.905,754.219944,-0.037349,53.259,11183.775146,0.047981,10.638,0.208329,0.009069
1957-01-01,29943.718022,6757.183908,0.020049,17.400,761.096724,0.009118,55.219,11571.193716,0.034641,10.949,-0.008021,0.000871
1958-01-01,29707.327613,6846.835727,0.013268,17.824,703.105546,-0.076194,56.222,11946.345811,0.032421,11.220,-0.146485,-0.004951
1959-01-01,31784.913478,7155.126050,0.045027,17.850,785.923138,0.117788,57.115,12544.310390,0.050054,11.521,0.367686,0.028656
1960-01-01,32600.949690,7273.985495,0.016612,18.063,801.859435,0.020277,56.899,13036.106883,0.039205,11.826,0.072226,0.020702
1961-01-01,33437.016772,7420.585479,0.020154,18.139,774.959241,-0.033547,57.043,13513.915427,0.036653,12.037,-0.042654,0.019670
1962-01-01,35485.104883,7649.084329,0.030793,18.238,863.444351,0.114180,57.288,14153.118630,0.047300,12.265,0.226254,0.022421
1963-01-01,37030.963170,7814.050709,0.021567,18.419,942.980855,0.092115,57.507,14756.440093,0.042628,12.461,-0.123818,0.023107


In [32]:
# Exportanto dados
dta2.to_csv('dataset_anual.csv')
files.download('dataset_anual.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>